In [2]:
library(Seurat)
seurat = readRDS('../initialseurat.RDS')

In [5]:
seurat = UpdateSeuratObject(seurat)
seurat@meta.data$letter = substring(seurat@meta.data$orig, 1, 1)
table(seurat@meta.data$letter)

Updating from v2.X to v3.X

Validating object structure

Updating object slots

Ensuring keys are in the proper strucutre

Ensuring feature names don't have underscores or pipes

Object representation is consistent with the most current Seurat version




    C     P 
 5191 35829 

In [6]:
resultsA = read.table('210514Lov.csv', header = 1, stringsAsFactors = FALSE, sep = ',', row.names = 1)
resultsB = read.table('211130Lov.csv', header = 1, stringsAsFactors = FALSE, sep = ',', row.names = 1)[6:11]

colnames(resultsB)[1:5] = colnames(resultsA)[1:5]
results = rbind(resultsA, resultsB)

In [7]:
seurat$id = paste0(seurat$orig, seurat$bc)
seurat = subset(seurat, cells = colnames(seurat)[seurat$id %in% rownames(results)])

In [8]:
hash_mat = results[seurat$id,]
bcs_to_names = colnames(seurat)
names(bcs_to_names) = seurat$id
rownames(hash_mat) = bcs_to_names[rownames(hash_mat)]

In [9]:
seurat[['HTO']] = CreateAssayObject(counts = t(hash_mat))
#seurat = subset(seurat, cells =colnames(seurat)[seurat$n_genes > 1000])

In [10]:
seurat = subset(seurat, cells = colnames(seurat)[seurat@meta.data$nCount_HTO > 5 ])

In [33]:
library(MASS)
library(cluster)
library(outliers)
library(dplyr)
seurat$hash = NA
Idents(seurat) = 'orig'
for (curr in unique(seurat$orig)) {
    message(curr)
   sub =subset(seurat, idents =  curr)
   sub =subset(sub, nCount_HTO > 20)

    if (length(colnames(sub)) > 50) {
        
        sub[['HTO']] = CreateAssayObject(counts = t(hash_mat[colnames(sub),c(1:5)]))

        sub = NormalizeData(sub, assay = 'HTO', normalization.method = 'CLR', margin = 2)
        
        

        DefaultAssay(sub) = 'HTO'
        data = GetAssayData(sub)

        # k medioids
        meds = pam(t(data), k = dim(data)[1])
        calls = t(data)
        cluster_data = meds$medoids %>% as.matrix()
        

        for (i in 1:dim(cluster_data)[2]) {
            
  

            curr_med = which.max(cluster_data[,i])
            negs = meds$clustering[meds$clustering != curr_med]
            neg_vals = data[i, names(negs)]
            pos_vals = data[i, meds$clustering == curr_med]
            pos_vals = pos_vals[order(pos_vals)] 
            done = FALSE
            thresh = min(pos_vals)
            # remove outliers and set threshold
            while(!done & length(pos_vals) > 5) {
                done = TRUE
                avg = mean(pos_vals)
                diff1 = avg - min(pos_vals)
                diff2 = max(pos_vals) - avg
                test = grubbs.test(pos_vals, opposite = diff1 < diff2)
                if (test$p.value < 0.05) {
                    pos_vals = pos_vals[-1]
                    done = FALSE
                }
                thresh = min(pos_vals)

            }
            print(thresh)
            if (thresh < .3) {
                thresh = 1.5
                message('danger')
            }
    if (curr == 'C1') {
                thresh = 1.2
            }
             if (curr == 'C4' & i == 4) {
                thresh = 1
            }
             if (curr == 'C8') {
                thresh = 1
            }
            
             if (curr == 'P4') {
                thresh = 1.3
            }
            
            calls[,i] = as.numeric(calls[,i] >= thresh)
            assign = apply(calls, 1, which.max)
            assign = paste0( 'HTO', assign)
            rsums = rowSums(calls)
            
        
            
            assign[rsums > 1] = 'Doublet'
            assign[rsums == 0] = 'Negative'
 
            seurat@meta.data[colnames(sub), 'hash'] = assign

            pheatmap::pheatmap(t(data), annotation_row = data.frame(row.names = colnames(data), id = factor(assign)), filename = paste0(curr, '.png'), show_rownames = FALSE)

        }
    }
}

C1

Normalizing across cells



[1] 1.287927
[1] 1.573499
[1] 1.287425
[1] 0.5094153
[1] 0


danger

C4

Normalizing across cells



[1] 1.001571
[1] 1.389664
[1] 0.6360292
[1] 1.172962
[1] 1.548001


C8

Normalizing across cells



...

danger

P5

Normalizing across cells



[1] 0.8665347
[1] 1.554289
[1] 0.9897035
[1] 1.345608
[1] 0.5223486


P6

Normalizing across cells



[1] 1.536691
[1] 1.799249
[1] 1.708771
[1] 1.720002
[1] 0


danger



In [34]:
saveRDS(seurat@meta.data, 'hashed_seurat_meta.RDS')
#saveRDS(seurat, 'hashed_seurat.RDS')
